In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
train_data = pd.read_csv("../input/train/train.csv")
test_data = pd.read_csv("../input/test/test.csv")

# Any results you write to the current directory are saved as output.

In [ ]:
train_data.describe()
#how is age 255? outlier
#what about max fee? 3000?
#count is equal across all columns, that means no missing values
list(test_data)

In [ ]:
#features to consider: number of dogs in image, age, breed, mixed breed?, quantity, fee, state
#(definitely image is part but thats a separate analysis)

boxplot=train_data.boxplot(column='Age')
boxplot1=train_data.boxplot(column='Quantity')
#to remove outliers for age
#train_age_out = train_data[~((train_data < (Q1 - 1.5 * IQR)) |(train_data > (Q3 + 1.5 * IQR))).any(axis=1)]
#train_age_out 


In [ ]:
# dropping ALL duplicate values 
train_data.drop_duplicates(subset ="PetID", 
                     keep = False, inplace = True) 
train_data.describe()

In [ ]:
from sklearn.model_selection import train_test_split

y = train_data.AdoptionSpeed
features = ['Age','Breed1','Breed2','Fee']
X = train_data[features]

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
#print(train_X, val_X, train_y, val_y)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X=train_X, val_X=val_X, train_y=train_y, val_y=val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [ ]:
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [30,36,40]:
    my_mae = get_mae(max_leaf_nodes)
    print(max_leaf_nodes, my_mae)

In [ ]:
#predict
import math
model = DecisionTreeRegressor(max_leaf_nodes=36, random_state=0)
model.fit(train_X, train_y)

test_predictions = model.predict(test_data[features])
test_predictions = np.round(test_predictions)
test_predictions = test_predictions.astype(int)

prediction_df = pd.DataFrame({'PetID' : test_data['PetID'],
                              'AdoptionSpeed' : test_predictions})
prediction_df.head()
#est_predictions

prediction_df.to_csv('submission.csv', index=False)